In this project you are trying out for an online marketplace company. The evaluation cover 3 important aspects of the daily job there that you need to know in order to get the position. 

The name of the database you will be working with is: data-analyst-web-store



Step 1:

Queries to know the data:

1. How many customers are there and what is the gender ratio?

2. How many items are there? Do you see any phenomena? (notice that item_id can be different)

3. What is the total amount of orders?

4. How many users were registered in July – December 2019 on the YouTube campaign?

5. How many users from the US bought the book Rich Dad? What is their share out of everyone who bought this book? And what is their share out of all US sales?

6. What is the most ordered item by quantity and what is the item with the highest total revenue? 

7. What is the share of total revenue, orders and users registration for each country, And how this ratio changes from 2019 to 2020 (show results for both years)

8. What is the total of shipments for each category in the data?

9. What is the average of the first order of each customer?

10. Does the membership split equally between gender? Do we get the same result per different countries in the data? Check only for English speaking countries (US, CA and EN)

11. What is the most successful campaign in terms of registrations? And what is the most successful campaign in terms of total revenue brought in?
Step 2:

1. Create a flat table per each order (flat – for every order bring every relevant information). For example: users registration, country, category etc. Save it to csv or excel file. PLEASE  validate the data.

2. Read the extra text file that can be find here to your current excel

In [ ]:
db_config = {'user': 'practicum_student',         # username             
                      'pwd': 's65BlTKV3faNIGhmvJVzOqhs', # password             
                      'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',       #host URL      
                      'port': 6432,              # connection port             
                      'db': 'data-analyst-web-store '}          # the name of the database